# Regression Model B: Field 4

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 4 (F4)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E4" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E4" ),]
dE4=d4[which(d4$Estate=="E4" ),]
dE5=d5[which(d5$Estate=="E4" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,manuring.time,manuring.cost,prun.time,prun.cost,
17,1,1,1,1,1,1,1,1,1,1,1,0
12,1,1,1,1,1,1,1,1,1,0,0,2
4,1,1,1,1,1,1,1,0,0,1,1,2
11,1,1,1,1,1,1,1,0,0,0,0,4
,0,0,0,0,0,0,0,15,15,23,23,76


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost
  3   3  prun.time  prun.cost  manuring.time  manuring.cost
  3   4  prun.time  prun.cost  manuring.time  manuring.cost
  3   5  prun.time  prun.cost  manuring.time  manuring.cost
  4   1  prun.time  prun.cost  manuring.time  manuring.cost
  4   2  prun.time  

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            23            23            15            15 
      pd.time       pd.cost 
            0             0 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost 
            2             3             4             5 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1             1             1       1
prun.cost          1         1  

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper")

## Train and test data

Split the dataset into train and test sets. After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

In this case, the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 


In [10]:
train = df2[df2$year<=2016,]
test = df2[df2$year==2017,]
dim(train)
dim(test)

[1] 32 11

[1] 12 11

## Regression

Fit the data using multiple regression for all variables. 

In [11]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-195.65  -33.24  -16.16   41.59  155.13 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)  
(Intercept)    4.521e+02  1.840e+02   2.457   0.0220 *
rainfall      -7.803e-03  4.476e-02  -0.174   0.8631  
foliar        -1.257e+01  5.561e+00  -2.260   0.0336 *
prun.time     -2.612e+01  3.924e+01  -0.666   0.5122  
prun.cost     -9.098e-04  9.026e-03  -0.101   0.9206  
manuring.time -1.978e+01  6.160e+01  -0.321   0.7511  
manuring.cost -8.188e-04  1.515e-03  -0.540   0.5941  
pd.time        2.338e+01  3.023e+01   0.773   0.4472  
pd.cost        5.689e-04  9.980e-03   0.057   0.9550  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 88.25 on 23 degrees of freedom
Multiple R-squared:  0.3854,	Adjusted R-squared:  0.1716 
F-statistic: 1.80

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [12]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,710.18722,710.18722,0.09119070,0.765382590
foliar,1,72626.55016,72626.55016,9.32552129,0.005630642
prun.time,1,16212.85382,16212.85382,2.08179121,0.162546830
prun.cost,1,13234.85943,13234.85943,1.69940556,0.205264016
manuring.time,1,1002.78301,1002.78301,0.12876110,0.722993062
manuring.cost,1,3066.76425,3066.76425,0.39378403,0.536497934
pd.time,1,5430.19804,5430.19804,0.69725778,0.412296740
pd.cost,1,25.30191,25.30191,0.00324886,0.955038468
Residuals,23,179122.49645,7787.93463,NA,NA


### Using stepwise

In [13]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=294.16
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- pd.cost        1        25 179148 292.17
- prun.cost      1        79 179202 292.18
- rainfall       1       237 179359 292.20
- manuring.time  1       803 179925 292.31
- manuring.cost  1      2274 181396 292.57
- prun.time      1      3451 182573 292.77
- pd.time        1      4657 183780 292.98
<none>                       179122 294.16
- foliar         1     39769 218892 298.58

Step:  AIC=292.17
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time

                Df Sum of Sq    RSS    AIC
- prun.cost      1       147 179295 290.19
- rainfall       1       224 179371 290.21
- manuring.time  1      1112 180260 290.37
- manuring.cost  1      2989 182136 290.70
- pd.time        1      5430 184578 291.12
- prun.time      1      6670 185817 291.34
<none>                   


Call:
lm(formula = yield ~ foliar + prun.time + pd.time, data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-189.397  -45.469   -1.343   45.917  163.548 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  383.691     74.264   5.167 1.76e-05 ***
foliar       -12.385      4.421  -2.802  0.00912 ** 
prun.time    -26.595     16.300  -1.632  0.11397    
pd.time       31.979     18.967   1.686  0.10291    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 81.42 on 28 degrees of freedom
Multiple R-squared:  0.3631,	Adjusted R-squared:  0.2949 
F-statistic: 5.321 on 3 and 28 DF,  p-value: 0.004981


   foliar prun.time   pd.time 
 1.037689  1.022109  1.024367 



Call:
lm(formula = yield ~ foliar + prun.time + pd.time, data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-189.397  -45.469   -1.343   45.917  163.548 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  383.691     74.264   5.167 1.76e-05 ***
foliar       -12.385      4.421  -2.802  0.00912 ** 
prun.time    -26.595     16.300  -1.632  0.11397    
pd.time       31.979     18.967   1.686  0.10291    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 81.42 on 28 degrees of freedom
Multiple R-squared:  0.3631,	Adjusted R-squared:  0.2949 
F-statistic: 5.321 on 3 and 28 DF,  p-value: 0.004981


## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [14]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

,Actual,Estimate
,<dbl>,<dbl>
33,347.478,236.8043
34,188.128,288.3769
35,486.897,275.9920
36,517.121,268.9905
37,469.213,367.8626
38,402.958,419.4352
39,413.973,343.0927
40,373.536,304.1124
41,298.210,291.7275


## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (  ) to know how close the data to the fitted regression line. 

In [15]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(k1)$r.squared)
fit

RMSE          R2 
113.7612882   0.3853712